In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from astropy.cosmology import LambdaCDM

from scipy.optimize import curve_fit

cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7)

In [9]:
# calling necessary class from ler package
from ler.gw_source_population import CBCSourceRedshiftDistribution

# uncomment the following line to see the docstring
# SourceGalaxyPopulationModel?

# class initialization
# default model "BBH popI/II Oguri2018"
cbc = CBCSourceRedshiftDistribution()

# list out the models for the merger rate density wrt redshift
print("\n available model list with it input parameters: \n", cbc.merger_rate_density_model_list)

z_to_luminosity_distance interpolator will be loaded from ./interpolator_pickle/z_to_luminosity_distance/z_to_luminosity_distance_3.pickle
differential_comoving_volume interpolator will be loaded from ./interpolator_pickle/differential_comoving_volume/differential_comoving_volume_1.pickle
merger_rate_density interpolator will be loaded from ./interpolator_pickle/merger_rate_density/merger_rate_density_2.pickle

 available model list with it input parameters: 
 {'merger_rate_density_bbh_popI_II_oguri2018': {'R0': 2.39e-08, 'b2': 1.6, 'b3': 2.0, 'b4': 30}, 'star_formation_rate_madau_dickinson2014': {'af': 2.7, 'bf': 5.6, 'cf': 2.9}, 'merger_rate_density_bbh_popIII_ken2022': {'n0': 1.92e-08, 'aIII': 0.66, 'bIII': 0.3, 'zIII': 11.6}, 'merger_rate_density_bbh_primordial_ken2022': {'n0': 4.4e-11, 't0': 13.786885302009708}}


In [5]:
# let's find mtot_max, let z_max = 2
# say ler default BNS mass model has a cut off 2.3 Mo
# mtot_max = 2.3*2*(1+2) = 13.8 

In [12]:
from ler.rates import LeR
ler = LeR(verbose=False)
ler.available_gw_prior_list_and_its_params

{'merger_rate_density': {'merger_rate_density_bbh_popI_II_oguri2018': {'R0': 2.39e-08,
   'b2': 1.6,
   'b3': 2.0,
   'b4': 30},
  'star_formation_rate_madau_dickinson2014': {'af': 2.7, 'bf': 5.6, 'cf': 2.9},
  'merger_rate_density_bbh_popIII_ken2022': {'n0': 1.92e-08,
   'aIII': 0.66,
   'bIII': 0.3,
   'zIII': 11.6},
  'merger_rate_density_bbh_primordial_ken2022': {'n0': 4.4e-11,
   't0': 13.786885302009708}},
 'source_frame_masses': {'binary_masses_BBH_popI_II_powerlaw_gaussian': {'mminbh': 4.98,
   'mmaxbh': 112.5,
   'alpha': 3.78,
   'mu_g': 32.27,
   'sigma_g': 3.88,
   'lambda_peak': 0.03,
   'delta_m': 4.8,
   'beta': 0.81},
  'binary_masses_BBH_popIII_lognormal': {'Mc': 30.0,
   'sigma': 0.3,
   'beta': 1.1},
  'binary_masses_BBH_primordial_lognormal': {'Mc': 30.0,
   'sigma': 0.3,
   'beta': 1.1},
  'binary_masses_BNS_gwcosmo': {'mminns': 1.0, 'mmaxns': 3.0, 'alphans': 0.0},
  'binary_masses_BNS_bimodal': {'w': 0.643,
   'muL': 1.352,
   'sigmaL': 0.08,
   'muR': 1.88,
   's

In [22]:
ler = LeR(npool=8, event_type = 'BNS', mtot_max=14, z_max = 2.0,
    # for source parameters
    source_priors=dict(
        merger_rate_density='merger_rate_density_bbh_popI_II_oguri2018',
        source_frame_masses='binary_masses_BNS_bimodal',
    ),

    # None means default values
    source_priors_params = dict( 
       merger_rate_density = None, 
       source_frame_masses=None
    ),
       
    # for lens parameters
    lens_functions=dict(
        strong_lensing_condition="rjs_with_cross_section_SIS",
        optical_depth="optical_depth_SIS_haris",
    ),
    lens_priors=dict(
        velocity_dispersion="velocity_dispersion_gengamma",
    ),
    lens_priors_params=dict(
        velocity_dispersion=dict(a=2.32 / 2.67, c=2.67)
    ),
    
    # for snr generation
    waveform_approximant = 'IMRPhenomXPHM',
    snr_type='inner_product',
    spin_zero=False,
    spin_precession=True,
    # change psd accordingly. I am providing default from bilby
    psds = dict( L1 = 'aLIGO_O4_high_asd.txt', H1 =  'aLIGO_O4_high_asd.txt'  ),
    # psds = dict( L1 = 'aligo_O4low_asd.txt', H1 =  'aligo_O4low_asd.txt'  ),
    # You don't need to provide ifos if you have given psds with detector name(s). Default detector orientation will be considered. For new detectors, you might want to pride detector location and orientation in ifos. 
    # ifos = ['L1'], 'H1'],
    verbose=False,
    )

In [23]:
ler.selecting_n_unlensed_detectable_events(size=5, batch_size=1000, snr_threshold=4.0, output_jsonfile='n_unlensed_detectable_bns_neha.json', meta_data_file='n_unlensed_detectable_bns_meta_neha.json', resume=True);

collected number of detectable events =  0


  0%|                                                                      | 0/1000 [00:00<?, ?it/s]

collected number of detectable events =  1
total number of events =  1000
total unlensed rate (yr^-1): 33.72002503359323


  0%|                                                                      | 0/1000 [00:00<?, ?it/s]

collected number of detectable events =  1
total number of events =  2000
total unlensed rate (yr^-1): 16.860012516796616


...

collected number of detectable events =  3
total number of events =  8000
total unlensed rate (yr^-1): 12.64500938759746


  0%|                                                                      | 0/1000 [00:00<?, ?it/s]

collected number of detectable events =  4
total number of events =  9000
total unlensed rate (yr^-1): 14.986677792708102


100%|███████████████████████████████████████████████████████████| 1000/1000 [00:15<00:00, 63.12it/s]

collected number of detectable events =  5
total number of events =  10000
total unlensed rate (yr^-1): 16.860012516796616
storing detectable unlensed params in ./ler_data/n_unlensed_detectable_bns_neha.json

 trmming final result to size=5


In [24]:
ler.selecting_n_lensed_detectable_events(size=5, batch_size=1000, snr_threshold=8.0, num_img=2, output_jsonfile='n_lensed_detectable_bns_neha.json', meta_data_file='n_lensed_detectable_bns_meta_neha.json', resume=True);

collected number of detectable events =  0


  0%|                                                                      | 0/1000 [00:00<?, ?it/s]

collected number of events =  1
total number of events =  1000
total lensed rate (yr^-1): 0.01091708098518182


  0%|                                                                      | 0/1000 [00:00<?, ?it/s]

collected number of events =  1
total number of events =  2000
total lensed rate (yr^-1): 0.00545854049259091


...

  0%|                                                                      | 0/1000 [00:00<?, ?it/s]

collected number of events =  1
total number of events =  13000
total lensed rate (yr^-1): 0.0008397754603986016


  0%|                                                                      | 0/1000 [00:00<?, ?it/s]

collected number of events =  1
total number of events =  14000
total lensed rate (yr^-1): 0.0007797914989415586


 44%|██████████████████████████▏                                 | 436/1000 [00:08<00:06, 92.16it/s]